In [2]:
import os
import mne
import pickle
from src.params import MRI_PATH, SUBJ_CLEAN, RESULT_PATH
from src.utils import get_bids_file
from src.config import subjects_dir
import numpy as np
from scipy import stats as stats
from mne.stats import spatio_temporal_cluster_1samp_test, summarize_clusters_stc

## Test average condition per subject

In [ ]:
####### PREPARE DATA #######

# Loop through subjects
all_subject_stcs_cond1 = []
all_subject_stcs_cond2 = []

all_stc_cond1 = []
all_stc_cond2 = []

group_all_stc_cond1 = []
group_all_stc_cond2 = []

task = 'LaughterActive'
cond1_name = 'LaughReal'
cond2_name = 'LaughPosed'

for subj in ['01', '02']:
    _, epo_path = get_bids_file(RESULT_PATH, task=task, subj=subj, stage="AR_epo")
    epochs = mne.read_epochs(epo_path)

    # Find condition related to epoch
    cond1_epo = []
    cond2_epo = []
    for i, epo_arr in enumerate(epochs.events): 
        if epo_arr[2] == 3 :                
            cond1_epo.append(i)
        elif epo_arr[2] == 2 : 
            cond2_epo.append(i)

    cond1 = [str(nb).zfill(3) for nb in cond1_epo]
    cond2 = [str(nb).zfill(3) for nb in cond2_epo]
    print('Cond1', len(cond1))
    print('Cond2', len(cond2))

    for epo1 in cond1 : 
        stc_path_cond1 = os.path.join(MRI_PATH, f'sub-{subj}', 'src',
                                      f'{epo1}_MNE_source-stc.h5')
        
        stc_cond1 = mne.read_source_estimate(stc_path_cond1)
        all_stc_cond1.append(stc_cond1)

    for epo2 in cond2 : 
        stc_path_cond2 = os.path.join(MRI_PATH, f'sub-{subj}', 'src',
                                      f'{epo2}_MNE_source-stc.h5')
        
        stc_cond2 = mne.read_source_estimate(stc_path_cond2)
        all_stc_cond2.append(stc_cond2)

    print(len(all_stc_cond1)) 
    print(len(all_stc_cond2))  

    ######## AVERAGE SIGNAL PER CONDITION FOR 1 SUBJ #########
    for subject_stcs1 in all_stc_cond1 : 

        # Average the SourceEstimates for the current subject
        avg_data_cond1 = np.mean([s.data for s in all_stc_cond1], axis=0)
        avg_stc_cond1 = mne.SourceEstimate(avg_data_cond1, 
                                           vertices=stc_cond1.vertices, 
                                           tmin=stc_cond1.tmin, 
                                           tstep=stc_cond1.tstep, 
                                           subject='sub-all')

        # Append to the list for all subjects
        all_subject_stcs_cond1.append(avg_stc_cond1)

    print('All subj cond1 :', len(all_subject_stcs_cond1))


    for subject_stcs2 in all_stc_cond2 : 

        # Average the SourceEstimates for the current subject
        avg_data_cond2 = np.mean([s.data for s in all_stc_cond2], axis=0)
        avg_stc_cond2 = mne.SourceEstimate(avg_data_cond1, 
                                           vertices=stc_cond2.vertices, 
                                           tmin=stc_cond2.tmin, 
                                           tstep=stc_cond2.tstep, 
                                           subject='sub-all')

        # Append to the list for all subjects
        all_subject_stcs_cond2.append(avg_stc_cond2)

    print('All subj cond2 :', len(all_subject_stcs_cond2))

    # Average 1 subject CONDITION 1
    group_avg_data_cond1 = np.mean([s.data for s in all_subject_stcs_cond1], axis=0)
    group_avg_stc_cond1 = mne.SourceEstimate(group_avg_data_cond1, 
                                             vertices=avg_stc_cond1.vertices, 
                                             tmin=avg_stc_cond1.tmin, 
                                             tstep=avg_stc_cond1.tstep, 
                                             subject=avg_stc_cond1.subject)

    # Average 1 subject CONDITION 2
    group_avg_data_cond2 = np.mean([s.data for s in all_subject_stcs_cond2], axis=0)
    group_avg_stc_cond2 = mne.SourceEstimate(group_avg_data_cond2, 
                                             vertices=avg_stc_cond2.vertices, 
                                             tmin=avg_stc_cond2.tmin, 
                                             tstep=avg_stc_cond2.tstep, 
                                             subject=avg_stc_cond2.subject)
    
    group_all_stc_cond1.append(group_avg_stc_cond1.data)
    group_all_stc_cond2.append(group_avg_stc_cond2.data)
    
print(np.array(group_all_stc_cond1).shape)
print(np.array(group_all_stc_cond2).shape)
# SAVE STC GROUPE !!!


Reading /home/claraelk/scratch/laughter_data/results/meg/reports/sub-01/sub-01_task-LaughterActive_run-all_AR_epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 7 columns
296 matching events found
No baseline correction applied
0 projection items activated
Cond1 73
Cond2 74
73
74
All subj cond1 : 73
All subj cond2 : 74
Reading /home/claraelk/scratch/laughter_data/results/meg/reports/sub-02/sub-02_task-LaughterActive_run-all_AR_epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    1500.00 ms
        0 CTF compensation matrices available
Adding metadata with 7 columns
295 matching events found
No baseline correction applied
0 projection items activated
Cond1 75
Cond2 72
148
146
All subj cond1 : 221


## Import data averaged per subject for each condition

In [3]:
contrast_data = []
group_all_stc_cond1 = []
group_all_stc_cond2 =[]

task = 'LaughterActive'
cond1_name = 'LaughReal'
cond2_name = 'LaughPosed'
conditions = cond1_name +'-'+ cond2_name

for subj in SUBJ_CLEAN :
    filename_contrast, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-contrast', condition=conditions)
    filename_cond1, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-contrast', condition=cond1_name)
    filename_cond2, _ = get_bids_file(RESULT_PATH, subj=subj, task=task, stage='erp-morph-contrast', condition=cond2_name)

    save_contrasts = os.path.join(MRI_PATH, 'sub-all', filename_contrast)

    save_cond1 = os.path.join(MRI_PATH, 'sub-all', filename_cond1)
    save_cond2 = os.path.join(MRI_PATH, 'sub-all', filename_cond2)

    with open(save_contrasts, 'rb') as f:
        contrast_subj = pickle.load(f)  

    with open(save_cond1, 'rb') as f:
        group_all_stc_cond1_subj = pickle.load(f)  

    with open(save_cond2, 'rb') as f:
        group_all_stc_cond2_subj = pickle.load(f) 
   
    contrast_data.append(contrast_subj)
    group_all_stc_cond1.append(group_all_stc_cond1_subj[0])
    group_all_stc_cond2.append(group_all_stc_cond2_subj[0])

In [4]:
contrast = np.array(group_all_stc_cond1)-np.array(group_all_stc_cond2) 
print(contrast.shape)

(27, 20484, 2401)


In [15]:
nan_mask = np.isnan(contrast)
if np.any(nan_mask):
    print("Warning: NaN values found in the data. Please address and rerun the analysis.")

In [5]:
contrast

array([[[ 1.53779361e-12,  3.81587687e-12,  2.63271323e-12, ...,
          1.10993472e-12,  1.43488986e-12, -3.59507710e-14],
        [ 8.45745976e-12,  4.79072361e-12, -1.15936286e-12, ...,
         -3.54733353e-12, -2.08827148e-12,  3.84516369e-13],
        [-2.16348922e-13,  1.62012105e-12,  2.76746839e-12, ...,
          4.35638990e-12,  5.31320680e-12,  4.90813267e-12],
        ...,
        [-5.44717023e-12,  6.50602911e-12,  1.56201246e-11, ...,
         -5.68071313e-12, -7.49380801e-12, -4.72773386e-12],
        [ 6.59459740e-13,  8.36739926e-12,  1.36615135e-11, ...,
         -5.16555219e-13, -2.40325228e-12, -2.59317666e-12],
        [ 7.82818872e-12,  1.04150593e-11,  1.02371069e-11, ...,
          3.91017450e-12,  3.46622077e-12,  2.08843185e-12]],

       [[-3.11219823e-13, -2.13149289e-14,  6.44996106e-13, ...,
          2.15698986e-12,  2.76738064e-12,  1.52956718e-12],
        [-1.51869318e-12, -3.05755996e-12, -2.48448956e-12, ...,
          3.80789912e-12,  2.34643991e

In [6]:
print("Min value:", np.min(contrast))
print("Max value:", np.max(contrast))


Min value: -2.0679515313825692e-25
Max value: 2.0679515313825692e-25


## Compute adjacency

In [18]:
# Read the source space we are morphing to
src_fname = '/home/claraelk/scratch/laughter_data/results/mri/anat/subjects/fsaverage/bem/fsaverage-5-src.fif'
fname_inv = os.path.join(MRI_PATH, 'sub-01', 'src', 'sub-01-mag-oct6-inv.fif')
inverse_operator = mne.minimum_norm.read_inverse_operator(fname_inv)

src = mne.read_source_spaces(src_fname)
fsave_vertices = [s["vertno"] for s in src]
morph_mat = mne.compute_source_morph(
    src=inverse_operator["src"],
    subject_to="fsaverage",
    spacing=fsave_vertices,
    subjects_dir=subjects_dir,
).morph_mat

n_vertices_fsave = morph_mat.shape[0]


Reading inverse operator decomposition from /home/claraelk/scratch/laughter_data/results/mri/sub-01/src/sub-01-mag-oct6-inv.fif...
    Reading inverse operator info...
    [done]
    Reading inverse operator decomposition...
    [done]
    270 x 270 full covariance (kind = 1) found.
    Noise covariance matrix read.
    24585 x 24585 diagonal covariance (kind = 2) found.
    Source covariance matrix read.
    24585 x 24585 diagonal covariance (kind = 6) found.
    Orientation priors read.
    24585 x 24585 diagonal covariance (kind = 5) found.
    Depth priors read.
    Did not find the desired covariance matrix (kind = 3)
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
    Source spaces transformed to the inverse solution coordinate

'\n# We have to change the shape for the dot() to work properly\nX = X.reshape(n_vertices_sample, n_times * n_subjects * 2)\nprint("Morphing data.")\nX = morph_mat.dot(X)  # morph_mat is a sparse matrix\nX = X.reshape(n_vertices_fsave, n_times, n_subjects, 2)\n'

In [ ]:
n_subjects = 27


# We have to change the shape for the dot() to work properly
group_cond1 = np.array(group_all_stc_cond1)
group_cond1 = group_cond1.reshape(n_vertices_sample, n_times * n_subjects)
print("Morphing data.")
group_cond1 = morph_mat.dot(group_cond1)  # morph_mat is a sparse matrix
group_cond1 = group_cond1.reshape(n_vertices_fsave, n_times, n_subjects)

In [19]:
print(n_vertices_fsave)

20484


In [92]:
file_src = 'sub-02-mag-oct6-inv.fif'
path_src = os.path.join(MRI_PATH, 'sub-02', 'src', file_src)

In [6]:
src_fname = '/home/claraelk/scratch/laughter_data/results/mri/anat/subjects/fsaverage/bem/fsaverage-5-src.fif'

src = mne.read_source_spaces(src_fname)
print(type(src))

    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    Distance information added...
    [done]
    2 source spaces read
<class 'mne.source_space.SourceSpaces'>


In [7]:
# Find adjacency
print("Computing adjacency.")
adjacency = mne.spatial_src_adjacency(src, dist=None)

Computing adjacency.
-- number of adjacent vertices : 20484


In [8]:
print(adjacency.shape)

(20484, 20484)


## Compute statistics

In [14]:
# Note that X needs to be a multi-dimensional array of shape
# observations (subjects) × time × space, so we permute dimensions
X = np.transpose(contrast, [0, 2, 1])
print(X.shape)
n_subjects = len(contrast)
print(n_subjects)

# Here we set a cluster forming threshold based on a p-value for
# the cluster based permutation test.
# We use a two-tailed threshold, the "1 - p_threshold" is needed
# because for two-tailed tests we must specify a positive threshold.
p_threshold = 0.001
df = n_subjects - 1  # degrees of freedom for the test
t_threshold = stats.distributions.t.ppf(1 - p_threshold / 2, df=df)

# Now let's actually do the clustering. This can take a long time...
print("Clustering.")
T_obs, clusters, cluster_p_values, H0 = clu = spatio_temporal_cluster_1samp_test(
    X,
    adjacency=adjacency,
    n_jobs=-1,
    threshold=None,
    buffer_size=None,
    verbose=True,
    n_permutations=1024,
    tail=0,
    step_down_p = 0.05,
    check_disjoint=True,
    out_type='indices'
)

(27, 2401, 20484)
27
Clustering.
Using a threshold of 2.055529
stat_fun(H1): min=-6.911754 max=6.263853
2 disjoint adjacency sets found
Running initial clustering …
Found 38009 clusters


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

KeyboardInterrupt: 

In [17]:
print('Loading cluster')
save_cluster_stats, _ = get_bids_file(RESULT_PATH, stage = "erp-source", task=task, measure="Ttest-clusters", condition = conditions)

path_save_cluster = os.path.join(MRI_PATH, 'sub-all', save_cluster_stats)

with open(path_save_cluster, 'rb') as f:
    clu = pickle.load(f)  

Loeading cluster


In [21]:
T_obs, clusters, cluster_p_values, H0 = clu 

# Select the clusters that are statistically significant at p < 0.05
good_clusters_idx = np.where(cluster_p_values < 0.05)[0]
good_clusters = [clusters[idx] for idx in good_clusters_idx]
print(good_clusters)

[]


In [23]:
print(cluster_p_values.shape)

(38009,)


In [24]:
print(np.unique(cluster_p_values))

[0.22851562 0.34960938 0.57421875 0.609375   0.98339844 0.99511719
 0.99707031 0.99804688 0.99902344 1.        ]


In [25]:
print(good_clusters_idx)

[]
